#### Backward generation(BWD)
- 20 data

1. generate data after integral

In [1]:
import random
from  random_unary_binary_tree import random_binary_trees
from  random_unary_binary_tree import traverse_unary_binary_prefix 
from infix_prefix import prefix_to_infix
from infix_prefix import infix_to_prefix
from sympy.parsing.sympy_parser import parse_expr

In [2]:
from __future__ import division
from sympy import *
import numpy as np

In [3]:
x = symbols('x', real=True)

In [4]:
sequence = []

i = 0
while i<=5:
    n = random.randint(1, 11)
    print(n)
    #n = 10
    root = random_binary_trees(n)
    result = traverse_unary_binary_prefix(root)
    if 'x' not in result:
        continue
    result_infix = prefix_to_infix(result)
    result_expr = " ".join(result_infix)
    result_simp = simplify(parse_expr(result_expr, local_dict={'x': x}))
    result_simp_prefix = infix_to_prefix(result_simp)
    print(result_simp_prefix)
    if 'x' not in result_simp_prefix or not result_simp_prefix:
        continue
    expression = simplify(diff(result_simp, x))
    expression_prefix = infix_to_prefix(expression)
    sequence.append([expression_prefix, result_simp_prefix])
    i += 1

4
['*', '-1', 'atanh', '+', '9', '*', '3', 'x']
3
9
5
5
['*', '-1', 'atan', '+', '1', 'x']
6
['*', 'pow', 'cos', '*', 'asin', '2', 'cos', '1', '-1', '+', '5', 'x']
3
6
11
7
9
['+', '*', '-1', 'x', '*', '/', '5', '4', 'pow', 'sin', '+', '5', 'cosh', 'cos', '10', '-1']
3
11
['+', '6', '+', '*', '-1', 'pow', '3', '/', '1', '2', '*', '3', '*', 'x', 'log', 'cosh', 'cos', '5']
5
3
8
['+', '2', 'cos', '+', '8', '*', 'exp', '4', 'sin', '+', '4', 'x']


### To do
- 1. change def function list to numpy
- 2. save sequence and label to csv file

test csvfile

In [5]:
import numpy as np

In [6]:
bwd_test = np.array(sequence)


In [61]:
bwd_test[0]

array([list(['*', '3', 'pow', '+', '-1', '*', '9', 'pow', '+', '3', 'x', '2', '-1']),
       list(['*', '-1', 'atanh', '+', '9', '*', '3', 'x'])], dtype=object)

In [7]:
bwd_test.shape

(6, 2)

In [8]:
pwd

'/Users/wenjuanyang/symbolic_math'

In [42]:
import numpy as np

np.savetxt('bwd_test.csv', bwd_test, delimiter=',', fmt="%s")

In [63]:
import csv
import ast
bwd_test2 = []
with open('bwd_test.csv', 'r') as f:
    for line in f:
        newline = ast.literal_eval(line)
        bwd_test2.append(list(newline))
    

In [64]:
bwd_test2 = np.array(bwd_test2)

In [65]:
bwd_test2.shape

(6, 2)

In [66]:
print(bwd_test2[0])

[list(['*', '3', 'pow', '+', '-1', '*', '9', 'pow', '+', '3', 'x', '2', '-1'])
 list(['*', '-1', 'atanh', '+', '9', '*', '3', 'x'])]
